Every CoV2 domain summary fasta

In [1]:
import pandas as pd
domains=pd.read_csv('nido-domains.csv')
domains.columns

Index(['genome_name', 'genome_length', 'domain_accession', 'strand', 'start',
       'end', 'hmmStart', 'hmmEnd', 'evalue', 'domain_annotation'],
      dtype='object')

(entry name for cov19:SARS-CoV-2||MN908947)

(entry for rdrp: PF00680)

In [2]:
used_domain=domains[domains['genome_name']=='SARS-CoV-2||MN908947']['domain_accession']

In [5]:
from glob import glob
from typing import Generator,Tuple
import json

def iter_match(genome_dict:dict)->Generator[Tuple[dict,dict],None,None]:
    '''
    same itermath as `extract_domains.py`
    put here for convienience
    '''
    for orf in genome_dict['results'][0]['openReadingFrames']:
        for match in orf['protein']['matches']:
            yield (orf,match)


def get_fasta(accession:str='PF00680',file_limit='nido_subset/*:genome.json'):
    '''
    cleansing needed
    '''
    o=[]
    o_dict={}
    b=0
    for gfile in glob(file_limit):
        # print('\n\n##'+gfile.split('/')[1].replace(':genome.json',''))
        genome_name=gfile.split('/')[1].replace(':genome.json','')
        genome_dict=json.load(open(gfile))
        genome_length=len(genome_dict['results'][0]['sequence'])
        for orf,match in iter_match(genome_dict):
            if match['signature']['accession']== accession:
                seq=orf['protein']['sequence']
                if len(match['locations'])==1:
                    _=match['locations'][0]
                    b,e=_['start'],_['end']
                    rdrp_seq=seq[b:e]
                    rdrp_name=genome_name
                    o.append(f'>{rdrp_name}\n{rdrp_seq}')
                    o_dict[rdrp_name]=rdrp_seq
                else:
                    for i,_ in enumerate(match['locations']):
                        b,e=_['start'],_['end']
                        rdrp_seq=seq[b:e]
                        rdrp_name=genome_name+'#'+str(i)
                        o.append(f'>{rdrp_name}\n{rdrp_seq}')
                        o_dict[rdrp_name]=rdrp_seq
    return o_dict,o

In [8]:
o={}
for i in used_domain:
    o[i]=get_fasta(i,'nido_subset/SARS-CoV-2||MN908947:genome.json')[0]
for k,v in o.items():
    with open(f'cov19-hits/{k}-cov2.fasta','w') as f:
        fastas='\n'.join([f'>{k1}\n{v1}' for k1,v1 in v.items()])
        f.write(fastas)

In [5]:
o={}
for i in used_domain:
    o[i]=get_fasta(i)[0]
for k,v in o.items():
    with open(f'cov19-hits/{k}.fasta','w') as f:
        fastas='\n'.join([f'>{k1}\n{v1}' for k1,v1 in v.items()])
        f.write(fastas)